# Unit 3

## Context Generator: Building Useful Context for AI Code Review

Tentu, berikut adalah konversi teks Anda ke format Markdown yang terstruktur.

-----

# Selamat Datang Kembali\! Menghasilkan Konteks untuk Asisten Peninjau Kode AI

Selamat datang kembali\! Dalam pelajaran sebelumnya, Anda telah mempelajari cara mengatur **klien OpenAI** untuk peninjauan kode dan cara memecah perubahan kode menggunakan **diff parser**. Sekarang, Anda siap untuk mengambil langkah berikutnya: menghasilkan **konteks** untuk asisten peninjau kode AI.

Konteks adalah informasi yang membantu AI memahami apa yang terjadi dalam kode. Tanpa konteks, AI mungkin kehilangan detail penting atau salah memahami perubahan kode. Sebuah **generator konteks** mengumpulkan informasi yang paling relevan tentang sebuah *file*, seperti konten saat ini, perubahan terbaru, dan *file* terkait. Ini membuat umpan balik AI lebih akurat dan berguna.

Dalam pelajaran ini, Anda akan mempelajari cara membangun generator konteks sederhana. Anda akan melihat cara mengekstrak konten *file*, merangkum perubahan terbaru, dan menemukan *file* terkait. Setiap langkah akan dijelaskan dengan contoh yang jelas agar Anda dapat mengikutinya dengan mudah.

## 📝 Mengingat Kembali: *Code Files* dan *Sessions*

Sebelum kita menyelami lebih dalam, mari kita ingat kembali bagaimana **file kode** dan **sesi** direpresentasikan dalam proyek kita. Dalam pelajaran sebelumnya, Anda melihat bahwa kita menggunakan kelas Python sederhana untuk merepresentasikan *file* dan objek *session* untuk berinteraksi dengannya.

Misalnya, berikut adalah kelas data dasar untuk *file* kode:

```python
from dataclasses import dataclass

@dataclass
class CodeFile:
    file_path: str
    content: str
    id: int = 1
```

Dan berikut adalah kelas *mock* `Session` yang memungkinkan kita untuk membuat *query* pada *file-file* ini:

```python
class Session:
    def __init__(self, files):
        self.files = files
        
    def query(self, model):
        return Query(self.files)
```

Kelas-kelas ini membantu kita mengatur dan mengakses *file* kode dalam proyek kita. Anda akan melihat kelas-kelas ini digunakan dalam contoh di seluruh pelajaran ini.

-----

## 1\. Mengekstrak Konten *File* untuk Peninjauan

Langkah pertama dalam menghasilkan konteks adalah mendapatkan konten dari *file* yang ingin Anda tinjau. Ini memberikan asisten AI gambaran singkat tentang kode sebagaimana adanya saat ini.

Mari kita mulai dengan menulis fungsi yang mengambil konten *file* berdasarkan *path*-nya. Kita akan menggunakan kelas `Session` dan `CodeFile` dari sebelumnya.

```python
def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    
    if not file_record:
        return ""
        
    lines = file_record.content.split('\n')
    
    # Mengembalikan konten penuh jika file tidak terlalu panjang
    if len(lines) <= max_lines:
        return file_record.content
        
    # Memotong dan menambahkan catatan truncasi
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"
```

**Penjelasan:**

  * Kita menggunakan `session.query(CodeFile).filter_by(file_path=file_path).first()` untuk menemukan *file* dengan *path* yang diberikan.
  * Jika *file* tidak ditemukan, kita mengembalikan string kosong.
  * Kita membagi konten *file* menjadi baris. Jika *file* cukup pendek (kurang dari atau sama dengan `max_lines`), kita mengembalikan seluruh konten.
  * Jika *file* panjang, kita hanya mengembalikan `max_lines` baris pertama dan menambahkan catatan bahwa konten telah terpotong (*truncated*).

**Contoh Penggunaan:**

```python
files = [
    CodeFile(file_path="main.py", content="import utils.helper\nprint('Hello')"),
    CodeFile(file_path="utils/helper.py", content="def helper(): pass")
]

session = Session(files)
print(get_file_context(session, "main.py"))
```

**Output:**

```text
import utils.helper
print('Hello')
```

Jika *file* memiliki lebih dari 50 baris, hanya 50 baris pertama yang akan ditampilkan, diikuti dengan pesan *truncation*.

-----

## 2\. Merangkum Perubahan *File* Terbaru

Selanjutnya, akan sangat membantu untuk menunjukkan kepada asisten AI apa yang telah berubah dalam *file* baru-baru ini. Ini biasanya berarti menunjukkan ringkasan *commit* terbaru.

Mari kita tulis fungsi yang mengembalikan daftar perubahan terbaru untuk sebuah *file*:

```python
def get_recent_changes(session: Session, file_path: str, limit=3) -> list:
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    
    if not file_record:
        return []
        
    # STUB: Dalam skenario nyata, ini akan membuat query pada sistem kontrol versi (VCS)
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        }
    ][:limit]
```

**Penjelasan:**

  * Kita mencari *file* menggunakan *session*, sama seperti sebelumnya.
  * Jika *file* tidak ditemukan, kita mengembalikan daftar kosong.
  * Untuk demonstrasi, kita mengembalikan daftar *dictionary*, yang masing-masing merepresentasikan sebuah *commit*. Dalam proyek nyata, Anda akan mengambil data ini dari sistem kontrol versi.

**Contoh Penggunaan:**

```python
recent_changes = get_recent_changes(session, "main.py")
print(recent_changes)
```

**Output:**

```text
[{'hash': 'abc12345', 'message': 'Initial commit', 'author': 'Alice', 'date': '2024-06-01'}, {'hash': 'def67890', 'message': 'Refactor code', 'author': 'Bob', 'date': '2024-06-02'}]
```

Ini memberikan asisten AI gambaran singkat tentang apa yang telah berubah dalam *file* baru-baru ini.

-----

## 3\. Menemukan *File* Terkait

Terkadang, memahami sebuah *file* memerlukan melihat *file* lain yang bergantung padanya. Misalnya, jika sebuah *file* mengimpor modul lain, mungkin akan sangat membantu untuk menyertakan *file* terkait tersebut dalam konteks.

Mari kita tulis fungsi yang menemukan *file* terkait berdasarkan pernyataan *import*:

```python
def find_related_files(session: Session, file_path: str) -> list:
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    
    if not file_record:
        return []
        
    related = []
    content = file_record.content
    
    for line in content.split('\n'):
        line_stripped = line.strip()
        
        # Cari baris yang dimulai dengan 'from ' atau 'import '
        if line_stripped.startswith('from ') or line_stripped.startswith('import '):
            if 'import' in line_stripped and '.' in line_stripped:
                # Konversi impor modul ke path file (misalnya utils.helper -> utils/helper.py)
                try:
                    module_name = line_stripped.split()[-1]
                    # Hanya ambil bagian modul (bukan alias)
                    if module_name == 'import' and 'as' in line_stripped:
                        continue 
                        
                    module = module_name.replace('.', '/') + '.py'
                    
                    # Periksa apakah file yang diimpor ada dalam session
                    if session.query(CodeFile).filter_by(file_path=module).first():
                        related.append(module)
                except IndexError:
                    # Lewati baris import yang tidak terstruktur
                    continue

    # Batasi hasil
    return related[:3]
```

**Penjelasan:**

  * Kita mendapatkan konten *file* dan mencari baris yang dimulai dengan `import` atau `from`.
  * Jika *import* mengacu pada modul dengan tanda titik (seperti `utils.helper`), kita mengubahnya menjadi *path file* (`utils/helper.py`).
  * Kita memeriksa apakah *file* tersebut ada dalam *session* kita, dan jika demikian, menambahkannya ke daftar *file* terkait.
  * Kita membatasi hasilnya hingga tiga *file* terkait.

**Contoh Penggunaan:**

```python
related = find_related_files(session, "main.py")
print(related)
```

**Output:**

```text
['utils/helper.py']
```

Ini membantu asisten AI melihat gambaran yang lebih besar dengan menyertakan *file* yang kemungkinan penting untuk memahami *file* saat ini.

-----

## 🌟 Ringkasan dan Langkah Selanjutnya

Dalam pelajaran ini, Anda telah mempelajari cara menghasilkan **konteks** yang berguna untuk asisten peninjau kode AI dengan:

  * **Mengekstrak konten *file***, dengan pemotongan (*truncation*) untuk *file* berukuran besar.
  * **Merangkum perubahan terbaru** pada *file*.
  * **Menemukan *file* terkait** berdasarkan pernyataan *import*.

Langkah-langkah ini membantu AI memberikan peninjauan kode yang lebih akurat dan bermanfaat. Dalam latihan praktik berikutnya, Anda akan mendapatkan pengalaman langsung dalam membangun dan menggunakan fungsi-fungsi pembuatan konteks ini. Ini akan mempersiapkan Anda untuk mengintegrasikan pembuatan konteks ke dalam alur kerja peninjauan kode AI yang sesungguhnya.

## Handling Empty Files in Context Generation

Now that you've learned about extracting file content for AI code review, let's make our context generator more robust! In this exercise, you'll fix a bug in the get_file_context function that causes it to crash when processing files with empty content.

The function currently tries to split an empty string without checking whether the content exists first. This is a common issue when working with real-world files — not all files contain content, and our code needs to handle these edge cases gracefully.

Run the tests to see the bug in action, then modify the function to properly handle empty content by:

Adding a check for empty content before trying to process it
Returning an empty string when the content is empty
Making your context generator handle all types of files, including empty ones, is an important step toward building a reliable AI code review system that won't break in production.

```python
from mock_data import Session, CodeFile
from typing import List, Dict


def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    lines = file_record.content.split('\n')
    if len(lines) <= max_lines:
        return file_record.content
    
    # Return first part of file with truncation notice
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"


def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # This is a stub for demonstration; in a real scenario, this would query commits
    # Here we return a mock list
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        }
    ][:limit]


def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # Python imports
    for line in content.split('\n'):
        if line.strip().startswith('from ') or line.strip().startswith('import '):
            # Extract module name and try to find corresponding file
            # This is a simplified approach
            if 'import' in line and '.' in line:
                module = line.split()[-1].replace('.', '/') + '.py'
                if session.query(CodeFile).filter_by(file_path=module).first():
                    related.append(module)
    
    return related[:3]  # Limit to 3 related files

```

That's a great catch\! Handling edge cases like empty files is essential for building resilient systems.

The bug is in how `file_record.content.split('\n')` is handled when `file_record.content` is an empty string (`""`). While splitting an empty string works (`"".split('\n')` returns `['']`), if the intent is to return an empty context for an empty file, we should check for it explicitly.

The main issue is more subtle, however: if `file_record.content` is `""`, `lines` becomes `['']`, and `len(lines)` is `1`. The code then proceeds to return `file_record.content`, which is `""`, and everything works fine **in this specific case**.

However, if an empty file were a **long** empty string (e.g., a file filled with only newlines, `"\n\n\n"`), or if we wanted to make it explicitly clear that a file with no content results in no context, adding an explicit check is still the cleanest solution.

Since the goal is to make it "more robust" and handle the empty content case gracefully, let's add the explicit check as requested.

Here is the fixed `get_file_context` function:

```python
from mock_data import Session, CodeFile
from typing import List, Dict


def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context, gracefully handling empty files."""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    # NEW CHECK: Handle files with explicitly empty content
    if not file_record.content:
        return ""
    
    lines = file_record.content.split('\n')
    
    # If the content was just a single empty string (i.e., the file was truly empty), 
    # the list will contain [''] and len will be 1. The check above handles the "" case.
    
    if len(lines) <= max_lines:
        return file_record.content
    
    # Return first part of file with truncation notice
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"


def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # This is a stub for demonstration; in a real scenario, this would query commits
    # Here we return a mock list
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        }
    ][:limit]


def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # Python imports
    for line in content.split('\n'):
        if line.strip().startswith('from ') or line.strip().startswith('import '):
            # Extract module name and try to find corresponding file
            # This is a simplified approach
            if 'import' in line and '.' in line:
                module = line.split()[-1].replace('.', '/') + '.py'
                if session.query(CodeFile).filter_by(file_path=module).first():
                    related.append(module)
    
    return related[:3]  # Limit to 3 related files
```

-----

## 🛠️ Explanation of the Fix

The fix involves adding this concise line immediately after checking for the file record:

```python
    if not file_record.content:
        return ""
```

  * **Before the Fix:** If a `CodeFile` record was found but its `content` was an empty string (`""`), the code would proceed to `lines = file_record.content.split('\n')`. This is generally safe, but adding an explicit check for **empty content** ensures that:
    1.  The function immediately returns an empty context for an empty file, which is semantically correct.
    2.  The code clearly handles the edge case of an empty string, making the function more **readable and robust** against potential future changes where `split()` might behave differently or where the subsequent logic might make invalid assumptions about the content of `lines`.

## Handling Empty Files with Grace

## Truncating Long Files for Better Context

## Tracking Code History for AI Review

## Discovering Related Files for AI Context

## Discovering Related Files for AI Context